In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
#http://scikit-learn.org/stable/modules/feature_selection.html

In [10]:
path='C:/Users/Lenovo/Desktop/iadap/3-2/IR/project/GOT-character-s-death-Prediction-master/'
data1=pd.read_csv(path+'Formated data.csv')
data=pd.read_csv(path+'Formated_data_corrected.csv')

col_is_alive=pd.DataFrame(data1['isAlive'])
data=col_is_alive.join(data)

#data.info()
x,y=list(),list()

In [11]:
def category(data):
    data.title=data.title.astype('category')  
    data.Culture=data.Culture.astype('category')        
    data.name=data.name.astype('category')
    data.house=data.house.astype('category')
    
    cat_columns = data.select_dtypes(['category']).columns
    data[cat_columns] = data[cat_columns].apply(lambda x: x.cat.codes)
    
    unique,counts=np.unique(data['title'],return_counts=True)
    diction=dict(zip(unique,counts))
    data['title']=data['title'].apply(lambda x: 1/diction[x]) 
    return data

In [12]:
def create_x_y(data):
    x=data.loc[:,'name':]
    #print(x)
    y=data.loc[:,'isAlive']  
    #print(y)
    return x,y

In [13]:
data=category(data)    
x,y=create_x_y(data)  
y=np.array(y)
y.shape

(1946,)

In [14]:
print(x.shape)
x_new = SelectKBest(chi2, k=20).fit_transform(x, y)
print(x_new.shape)

(1946, 53)
(1946, 20)


In [15]:
print(x_new[0])

[  1.82700000e+03   1.00000000e+00   2.90000000e+01   1.97000000e+02
   0.00000000e+00   0.00000000e+00   6.05351171e-01   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]


In [16]:
y_old=y
clf = ExtraTreesClassifier()
clf = clf.fit(x, y)
print(clf.feature_importances_ )

model = SelectFromModel(clf, prefit=True)
x_new = model.transform(x)
x_new.shape

y_new=np.zeros((len(y_old),2))
for i in range(len(y_old)):
    if y_old[i]==1:
        y_new[i]=[1,0]
        #print(y_new[i])
    else:
        y_new[i]=[0,1]
        #print(y_new[i])

[  1.74981114e-01   5.86108623e-02   2.96831673e-02   8.41182713e-02
   1.06811524e-01   3.05032830e-02   2.43799865e-02   3.10447104e-02
   7.28468060e-02   4.09600973e-02   2.76897802e-02   3.08099390e-02
   2.29245526e-01   1.73082395e-03   1.04271495e-03   1.08466097e-03
   1.84475320e-04   9.07933328e-04   1.34011015e-03   1.23248349e-03
   4.95384376e-04   5.68253157e-04   1.03029746e-03   1.45038067e-03
   1.07175459e-03   3.53733377e-04   1.77048820e-03   9.80048105e-04
   3.58991285e-04   3.75840896e-04   3.80056577e-04   2.61090930e-04
   9.28816072e-04   1.77778711e-03   1.61354038e-03   0.00000000e+00
   4.71204493e-04   1.29018186e-05   2.28060203e-03   2.43961753e-03
   3.69042276e-04   7.08580452e-04   0.00000000e+00   3.66887211e-03
   2.01741354e-03   2.55733697e-03   3.98758131e-04   7.50855832e-04
   1.65301819e-03   4.52348450e-03   2.81084310e-03   6.53300598e-03
   6.17973026e-03]


In [17]:
y_old

array([0, 1, 1, ..., 1, 1, 0], dtype=int64)

In [18]:
y=y_new.reshape((y_new.shape[0],2))
x_train,x_test,y_train,y_test=train_test_split(x_new,y,test_size=0.2)
print(y_train.shape)
print(x_train.shape)

(1556, 2)
(1556, 13)


In [19]:
random_state = 42
np.random.seed(random_state)
tf.set_random_seed(random_state)                

In [20]:
def multilayer_perceptron(x, weights, biases, keep_prob):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, keep_prob)
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

In [21]:
n_hidden_1 = 38
n_input = x_train.shape[1]
n_classes = 2

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

keep_prob = tf.placeholder("float")


In [22]:
training_epochs = 1000
display_step = 100
batch_size = 32

x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [23]:
predictions = multilayer_perceptron(x, weights, biases, keep_prob)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)

In [24]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(len(x_train) / batch_size)
        x_batches = np.array_split(x_train, total_batch)
        y_batches = np.array_split(y_train, total_batch)
        for i in range(total_batch):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, cost], 
                            feed_dict={
                                x: batch_x, 
                                y: batch_y, 
                                keep_prob: 0.8
                            })
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))

Epoch: 0001 cost= 1194.289983431
Epoch: 0101 cost= 758.827219963
Epoch: 0201 cost= 403.400506814
Epoch: 0301 cost= 262.292869965
Epoch: 0401 cost= 105.365829746
Epoch: 0501 cost= 30.437450568
Epoch: 0601 cost= 7.227172511
Epoch: 0701 cost= 2.518039990
Epoch: 0801 cost= 1.683328706
Epoch: 0901 cost= 1.635202954
Optimization Finished!
Accuracy: 0.74359
